# Vamos a hacer unos modelos para la predicción de energía.  
Esto es casi igual que lo de la clasificación pero tenemos que hacerlo con las labels de energia.

In [1]:
#importamos librerias

import sys
sys.path.append('../src/CTA-data-analisis-library/')
import os 
import subprocess
from datetime import datetime
import numpy as np 
import glob
import matplotlib.pyplot as plt
import tensorflow as tf 
import psutil
import re
import random
import shutil
import pickle
from numba import cuda
import gc

#propias
import unzipdata_and_first_treatments as manipulate
import loaddata4use
import model_creation_functions as models

In [2]:
tf.config.threading.set_inter_op_parallelism_threads(4)

In [3]:
#enviroment variables
npy_final_dir="../datos/elementos_npy"
base_dir_elementos="../datos/elementos"
elements=['gamma', 'electron']

In [4]:
def runs_disponibles(npy_dir,elements):
    lista=[]
    for i in elements:
        npy_element_dir=os.path.join(npy_dir,"npy_"+i)
        runs=[int(re.search("run_([0-9]{3})_0\.npy",i).group(1)) for i in os.listdir(npy_element_dir) if re.search("run_([0-9]{3})_0\.npy",i)]
        lista.append(runs)
    return lista


def new_create_main_list_runs(number_runs_per_element,posibles_runs):
    #esto es aleatorio por defecto, porque es lo unico que necesito por ahora    
    final=[]
    for ind,lista_runs_element in enumerate(posibles_runs):
        final.append(random.sample(lista_runs_element,number_runs_per_element[ind]))
    return final

#tenemos que hacer un ligero cambio porque se estan cargando con los ejes cambiados
def cambiar_ejes_lista(lista):
    for i,j in enumerate(lista):
        lista[i]=np.swapaxes(j,1,2)
    return lista

def get_all_size(local_vars):
    #local_vars = list(locals().items())
    total=0
    for var, obj in local_vars:
        total+=sys.getsizeof(obj)
    total= total >> 20
    return total # its in Mb

In [5]:
chose_runs=runs_disponibles(npy_final_dir,elements)
chose_runs[0].remove(2)
chose_runs[0].remove(2)
chose_runs[0].remove(3)
chose_runs[0].remove(3)
chose_runs[0].remove(3)

In [6]:
opciones_filtros=[
    [[16,32],[64,128],[128,64],[64,32]],
    [[32,64],[64,128],[128,64,32]],
    [[16,32],[32,64],[64,32,16]]]

opciones_filtros_last=[
    [20,10],[20,5],[10,5]
]

In [10]:
#vamos a llevar un registros del punto en el que se esta quedando el modelo
#voy a hacer una callback que guarde el último modelo, y con ello
#vamos a ir guardando los ultimo datos de la ejecucion y si hay un problema, retornamos a ese punto

#si tenemos que retomar el entrenamiento, vamos a indicar que modelo usar

device = cuda.get_current_device()
file_number="007"
n=17 #repes de boostrap
#primer bucle para arquitecturas
for i,arch in enumerate(opciones_filtros):
    print(f"{i}: {arch} \n")

    modelo=models.model_multi_tel_energy(len_inputs=4,input_shapes=[(55,93,1)],filtros=arch,last_dense=opciones_filtros_last[i]) #no compila
    modelo.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),loss="mse",metrics=["mae","mape"])
    with open(f"../automat/logs/{file_number}_data_control_energy.txt","a") as registro:
        registro.write(f"Con arquitectura: {arch}, y uso de CPU por el modelo {tf.config.experimental.get_memory_info('CPU:0')['current']>>20}Mb, y memoria {get_all_size(list(locals().items()))} Mb \n")

    #segundo_bucle para boostrap
    for k in range(n):
        #modificamos el learning rate
        if k == 4:
            modelo.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),loss="mse",metrics=["mae","mape"])
        elif k == 10:
            modelo.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),loss="mse",metrics=["mae","mape"])
        elif k == 15:
            modelo.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-6),loss="mse",metrics=["mae","mape"])


        print(f"\n Boostrap {k+1} de {n}, y uso de CPU por el modelo {tf.config.experimental.get_memory_info('CPU:0')['current']>>20}Mb, y memoria {get_all_size(list(locals().items()))} Mb \n")

        list_runs=new_create_main_list_runs([15,60],chose_runs)#new_create_main_list_runs([2,6,6,6,6,6,6],chose_runs)
        with open(f"../automat/logs/{file_number}_data_control_energy.txt","a") as registro:
            registro.write(f"Boostrap {k+1} de {n},runs: {list_runs}, y uso de CPU por el modelo {tf.config.experimental.get_memory_info('CPU:0')['current']>>20}Mb, y memoria {get_all_size(list(locals().items()))} Mb \n")
        x_train_list,x_test_list,y_train_list,y_test_list=loaddata4use.load_dataset_energy(npy_final_dir,base_dir_elementos,elementos=['gamma', 'electron'],main_list_runs=list_runs,telescopios=[1,2,3,4],test_size=0.01,
            same_quant="same",verbose=True,fill=True,lower_energy_bound=1,upper_energy_bound=10)
        print(f"\n Despues de cargar las variables, en memoria {get_all_size(list(locals().items()))} Mb \n")

        x_train_list=cambiar_ejes_lista(x_train_list)
        x_test_list=cambiar_ejes_lista(x_test_list)

        
        hist=modelo.fit(x=x_train_list,y=y_train_list,epochs=7, validation_data=(x_test_list,y_test_list),batch_size=32)
        del x_train_list,x_test_list,y_train_list,y_test_list
        gc.collect()
        with open(f"../automat/logs/{file_number}_data_control_energy.txt","a") as registro:
            registro.write(f"Al borrar memoria nos quedan {get_all_size(list(locals().items()))} Mb \n")
        print(f"\n Al borrar memoria nos quedan {get_all_size(list(locals().items()))} Mb \n")
        modelo.save(f"../modelos/{file_number}_modelo_filtro_{i}_en_boostrap_stage_{k+1}_energy.h5")
        with open(f"../modelos/performances/{file_number}_history_modelo_filtro_{i}_en_boostrap_stage_{k+1}_energy.pickle","wb") as pick:
            pickle.dump(hist.history,pick)    
    del modelo 
    gc.collect()
    tf.keras.backend.clear_session()
    device.reset()

        


0: [[16, 32], [64, 128], [128, 64], [64, 32]] 


 Boostrap 1 de 5, y uso de CPU 19Mb, y memoria 0 Mb 

Load of names and common events
_______________
Element:  gamma  , Runs:  [15]  Shape of common events (common events, energies):  (4, 1176) (4, 1176)
Element:  electron  , Runs:  [16, 4]  Shape of common events (common events, energies):  (4, 232) (4, 232)
Element:  electron  , Runs:  [16, 4]  Shape of common events (common events, energies):  (4, 224) (4, 224)
_______________
Load of actual npy data:
_______________
49
Element:  gamma , Telescope:  1  Shape of loaded array (amount of images, size of images):  (49, 93, 55)
Energies vector shape: (49,)
Element:  electron , Telescope:  1  Shape of loaded array (amount of images, size of images):  (49, 93, 55)
Energies vector shape: (49,)
Final data shape (98, 93, 55, 1) Final energies shape:  (98,)
_____________________ 

Element:  gamma , Telescope:  2  Shape of loaded array (amount of images, size of images):  (49, 93, 55)
Element:  

INFO:tensorflow:Assets written to: ram://d927bcc8-5cc0-497d-97af-e92d963178ec/assets


INFO:tensorflow:Assets written to: ram://d927bcc8-5cc0-497d-97af-e92d963178ec/assets



 Boostrap 2 de 5, y uso de CPU 20Mb, y memoria 0 Mb 

Load of names and common events
_______________
Element:  gamma  , Runs:  [49]  Shape of common events (common events, energies):  (4, 1100) (4, 1100)
Element:  electron  , Runs:  [92, 79]  Shape of common events (common events, energies):  (4, 231) (4, 231)
Element:  electron  , Runs:  [92, 79]  Shape of common events (common events, energies):  (4, 243) (4, 243)
_______________
Load of actual npy data:
_______________
59
Element:  gamma , Telescope:  1  Shape of loaded array (amount of images, size of images):  (59, 93, 55)
Energies vector shape: (59,)
Element:  electron , Telescope:  1  Shape of loaded array (amount of images, size of images):  (59, 93, 55)
Energies vector shape: (59,)
Final data shape (118, 93, 55, 1) Final energies shape:  (118,)
_____________________ 

Element:  gamma , Telescope:  2  Shape of loaded array (amount of images, size of images):  (59, 93, 55)
Element:  electron , Telescope:  2  Shape of loaded ar

KeyboardInterrupt: 

In [15]:
hist.history()

TypeError: 'dict' object is not callable